O algoritmo a seguir fornece uma implementação para monitorar a entropia de Shannon de flags TCP em um fluxo de dados (data stream) em tempo real. Usando dataset rotulado com 5 flgs.

In [1]:
import pandas as pd
import numpy as np
import time
from collections import deque
import matplotlib.pyplot as plt  # Importar matplotlib para criar gráficos


In [2]:
# Função para calcular entropia de Shannon
def calcular_entropia(series):
    """Calcula a entropia de Shannon para uma série categórica."""
    counts = series.value_counts(normalize=True)  # Obter distribuição de probabilidades
    entropy = -np.sum(counts * np.log2(counts + 1e-10))  # Aplicar fórmula de Shannon
    return entropy

'''
def analisar_distribuicao(series):
    """
    Analisa a distribuição de uma série categórica.
    Imprime e retorna o total de amostras, os valores únicos e a contagem de cada valor único.
    """
    # Total de amostras
    total_amostras = len(series)
    
    # Valores únicos e suas contagens
    contagens = series.value_counts()
    valores_unicos = contagens.index.tolist()
    repeticoes = contagens.values.tolist()
    
    # Imprimir informações
    print(f"Total de amostras: {total_amostras}")
    print(f"Valores únicos: {valores_unicos}")
    print(f"Repetições: {repeticoes}")
    
    # Retornar resultados (opcional, caso você ainda precise dos dados)
    return {
        "total_amostras": total_amostras,
        "valores_unicos": valores_unicos,
        "repeticoes": repeticoes
    }'''

# Classe para monitoramento da entropia em tempo real (Data Stream)
class EntropyMonitor:
    def __init__(self, flags, window_size=100):
        self.flags = flags  # Nomes das colunas (flags TCP)
        self.window_size = window_size  # Tamanho da janela deslizante
        self.stream_data = deque(maxlen=window_size)  # Estrutura para armazenar dados

    def update_stream(self, new_data):
        """Atualiza a janela de dados com novas observações."""
        self.stream_data.append(new_data)

    def calculate_entropy(self):
        """Calcula a entropia das flags TCP dentro da janela de observação."""
        if len(self.stream_data) < 2:  # Evita cálculo com poucos dados
            return {flag: 0 for flag in self.flags}
        
        # Criar DataFrame da janela atual
        stream_df = pd.DataFrame(list(self.stream_data), columns=self.flags)
        
        # Calcular entropia das flags
        entropia_flags = {flag: calcular_entropia(stream_df[flag]) for flag in self.flags}
        return entropia_flags

In [ ]:
'''
# Carregar CSV
df = pd.read_csv('dataset_legitimo_malicioso.csv', sep=',')

# Selecionar colunas relevantes
flags = ['tcp.flags.ack', 'tcp.flags.syn', 'tcp.flags.fin']
df2 = df[['frame.time_relative', 'label'] + flags].copy()
df2 = df2.head(402513)  # garantir tamanho total

# Novo monitor
def novo_monitor():
    return EntropyMonitor(flags, window_size=600)

# Inicializar monitor e resultados
entropy_monitor = novo_monitor()
stream_results = []

# Índices das transições entre as partes
limite_parte1 = 140544  # início da parte 2
limite_parte2 = 261969  # início da parte 3

print("Iniciando monitoramento de entropia com resets entre as partes...\n")

# Loop de simulação do stream
for i in range(402513):
    # Reiniciar monitor nas transições
    if i == limite_parte1 or i == limite_parte2:
        entropy_monitor = novo_monitor()
    
    new_sample = df2.iloc[i].to_dict()
    entropy_monitor.update_stream(new_sample)

    entropia_atual = entropy_monitor.calculate_entropy()

    stream_results.append({
        "Iteração": i + 1,
        "frame.time_relative": new_sample['frame.time_relative'],
        "label": new_sample['label'],
        **entropia_atual
    })

# Criar e salvar o DataFrame com entropia
stream_df = pd.DataFrame(stream_results)
stream_df.to_csv("corrigido_entropia_legitimo_malicioso.csv", index=False)

print("\nMonitoramento finalizado com resets nas transições entre partes.")
'''

Iniciando monitoramento de entropia com resets entre as partes...


Monitoramento finalizado com resets nas transições entre partes.


flags+colunas nao processdas+tempo (nao ficou bom)

Limpeza do dataset com procesamento, dataset com aspas, conchetes e NaN

In [ ]:
'''
import pandas as pd
import numpy as np

# === PASSO 1: Carregar o CSV ===
df = pd.read_csv('dados_legitimos.csv', sep=',')

# Colunas de interesse
flags = ['tcp.flags.ack', 'tcp.flags.syn', 'tcp.flags.fin']
print(flags)
# Criar df2 com apenas tempo e flags
df2 = df[['frame.time_relative'] + flags].copy()

# Limitar a quantidade de amostras
df2 = df2.head(140544)

# === PASSO 1.2: Limpeza ===
# Remove aspas simples e colchetes em todo o DataFrame
df2 = df2.replace({r"'": "", r"\[": "", r"\]": ""}, regex=True)

# Substituir NaN e strings 'nan' por 0
df2 = df2.replace(['nan', np.nan], 0)

# Converter colunas de flags para float
df2[flags] = df2[flags].astype(float)

# Verificar o shape
print(df2.shape)

# Criar monitor de entropia
entropy_monitor = EntropyMonitor(flags, window_size=600)
'''

<>:1: SyntaxWarning: invalid escape sequence '\['
<>:1: SyntaxWarning: invalid escape sequence '\['
/tmp/ipykernel_9070/2414160219.py:1: SyntaxWarning: invalid escape sequence '\['
  '''


'\nimport pandas as pd\nimport numpy as np\n\n# === PASSO 1: Carregar o CSV ===\ndf = pd.read_csv(\'dados_legitimos.csv\', sep=\',\')\n\n# Colunas de interesse\nflags = [\'tcp.flags.ack\', \'tcp.flags.syn\', \'tcp.flags.fin\']\nprint(flags)\n# Criar df2 com apenas tempo e flags\ndf2 = df[[\'frame.time_relative\'] + flags].copy()\n\n# Limitar a quantidade de amostras\ndf2 = df2.head(140544)\n\n# === PASSO 1.2: Limpeza ===\n# Remove aspas simples e colchetes em todo o DataFrame\ndf2 = df2.replace({r"\'": "", r"\\[": "", r"\\]": ""}, regex=True)\n\n# Substituir NaN e strings \'nan\' por 0\ndf2 = df2.replace([\'nan\', np.nan], 0)\n\n# Converter colunas de flags para float\ndf2[flags] = df2[flags].astype(float)\n\n# Verificar o shape\nprint(df2.shape)\n\n# Criar monitor de entropia\nentropy_monitor = EntropyMonitor(flags, window_size=600)\n'

COM features no tempo originais e features selecionadas para entropia 7/05

In [ ]:

# === PASSO 1: Carregar o CSV ===
df = pd.read_csv('dataset_legitimo_malicioso.csv', sep=',')

# Separar o que é para calcular entropia
flags = ['tcp.flags.ack', 'tcp.flags.syn', 'tcp.flags.fin']

# Criar df2 com tempo + flags
#df2 = df[['frame.time_relative','label','tcp.len','tcp.time_delta','mqtt.len'] + flags].copy()
df2 = df[['frame.time_relative', 'label'] + flags].copy()

# Limitar a quantidade de amostras
df2 = df2.head(402513)

# Verificar o shape
print(df2.shape)

# Criar monitor de entropia só com as flags TCP
entropy_monitor = EntropyMonitor(flags, window_size=600)


"\n# === PASSO 1: Carregar o CSV ===\ndf = pd.read_csv('dataset_legitimo_malicioso.csv', sep=',')\n\n# Separar o que é para calcular entropia\nflags = ['tcp.flags.ack', 'tcp.flags.syn', 'tcp.flags.fin']\n\n# Criar df2 com tempo + flags\n#df2 = df[['frame.time_relative','label','tcp.len','tcp.time_delta','mqtt.len'] + flags].copy()\ndf2 = df[['frame.time_relative', 'label'] + flags].copy()\n\n# Limitar a quantidade de amostras\ndf2 = df2.head(402513)\n\n# Verificar o shape\nprint(df2.shape)\n\n# Criar monitor de entropia só com as flags TCP\nentropy_monitor = EntropyMonitor(flags, window_size=600)\n"

Extraçao normal

In [6]:
'''
# === PASSO 1: Carregar o CSV ===
df = pd.read_csv('dados_rotulados22.csv', sep=',')

# Definir as colunas 
#flags = ['tcp.srcport', 'tcp.dstport', 'tcp.len', 'tcp.flags.ack',
        #'tcp.flags.syn', 'tcp.flags.fin']

flags = ['frame.time_relative','frame.time_epoch','tcp.flags.ack','tcp.flags.syn', 'tcp.flags.fin', 'tcp.flags.urg']

# Criar um novo DataFrame apenas com essas colunas
df2 = df[flags].copy()

# Limitar a quantidade de amostras
df2 = df2.head(140544)

# Verificar o shape
print(df2.shape)

# Criar monitor de entropia para as flags TCP
entropy_monitor = EntropyMonitor(flags, window_size=600)  # Define o tamanho da janela deslizante.
#Quantidade de amostras que serão consideradas de cada vez para calcular a entropia.
'''

"\n# === PASSO 1: Carregar o CSV ===\ndf = pd.read_csv('dados_rotulados22.csv', sep=',')\n\n# Definir as colunas \n#flags = ['tcp.srcport', 'tcp.dstport', 'tcp.len', 'tcp.flags.ack',\n        #'tcp.flags.syn', 'tcp.flags.fin']\n\nflags = ['frame.time_relative','frame.time_epoch','tcp.flags.ack','tcp.flags.syn', 'tcp.flags.fin', 'tcp.flags.urg']\n\n# Criar um novo DataFrame apenas com essas colunas\ndf2 = df[flags].copy()\n\n# Limitar a quantidade de amostras\ndf2 = df2.head(140544)\n\n# Verificar o shape\nprint(df2.shape)\n\n# Criar monitor de entropia para as flags TCP\nentropy_monitor = EntropyMonitor(flags, window_size=600)  # Define o tamanho da janela deslizante.\n#Quantidade de amostras que serão consideradas de cada vez para calcular a entropia.\n"

In [7]:
print(df2)

        frame.time_relative  label  tcp.flags.ack  tcp.flags.syn  \
0                  0.000000      0            0.0            0.0   
1                  0.076191      0            0.0            1.0   
2                  0.076202      0            0.0            1.0   
3                  0.091916      0            0.0            1.0   
4                  0.091984      0            0.0            1.0   
...                     ...    ...            ...            ...   
402508          1064.991926      0            0.0            1.0   
402509          1065.028879      0            0.0            1.0   
402510          1065.028955      0            0.0            1.0   
402511          1065.032882      0            0.0            1.0   
402512          1065.032892      0            0.0            1.0   

        tcp.flags.fin  
0                 0.0  
1                 0.0  
2                 0.0  
3                 0.0  
4                 0.0  
...               ...  
402508         

Processamento para flags+colunas nao processadas 7/05

In [8]:
"""
# === PASSO 2: Simulação do Data Stream ===
num_iterations = 402513  # Número de iterações da simulação
#Simula pacotes (amostras) chegando,e essas amostras sao extraidas do dataset.
#Cada iteração do loop simula a chegada de um novo pacote (ou seja, uma nova amostra é processada).
stream_results = []

print("Monitorando entropia das flags TCP...\n")

for i in range(num_iterations):
    # Simular novos pacotes (extraindo amostras do conjunto de dados original)
    #new_sample = df2.sample(n=1).iloc[0].to_dict() #A mesma amostra pode ser escolhida mais de uma vez.
    #Em cada iteração, uma amostra é escolhida aleatoriamente do df2

    new_sample = df2.iloc[i].to_dict()  #Simula novos pacotes (extraindo amostras em oderm sequencial do dataset)
    entropy_monitor.update_stream(new_sample)
    
    # Calcular a entropia na janela atual
    entropia_atual = entropy_monitor.calculate_entropy()
    
    # Armazenar os resultados do stream
    #stream_results.append({"Iteração": i + 1, **entropia_atual})
    
    # Armazenar os resultados do stream (AGORA COM TEMPO REAL INCLUÍDO)
    stream_results.append({
        "Iteração": i + 1,
        "frame.time_relative": new_sample['frame.time_relative'],  # adiciona o tempo original
        "label": new_sample['label'],
        **entropia_atual
    })

    '''
    # Adiciona a iteração + tempos originais + entropia no registro
    stream_results.append({
        "Iteração": i + 1,
        "frame.time_relative": new_sample['frame.time_relative'],
        "label": new_sample['label'], 
        "tcp.len": new_sample['tcp.len'],  
        "tcp.time_delta": new_sample['tcp.time_delta'], 
        "mqtt.len": new_sample['mqtt.len'], 
        **entropia_atual
    })'''

    # Exibir resultados em tempo real
    print(f"Iteração {i+1}: {entropia_atual}")

    '''
    #Adcionado hj 21/03
    # Analisar a distribuição de cada flag na janela atual
    stream_df = pd.DataFrame(list(entropy_monitor.stream_data), columns=flags)
    distribuicoes = {flag: analisar_distribuicao(stream_df[flag]) for flag in flags}
    
    # Exibir informações da distribuição
    for flag in flags:
        print(f"  Flag: {flag}")
        print(f"    - Total de amostras: {distribuicoes[flag]['total_amostras']}")
        print(f"    - Valores únicos: {distribuicoes[flag]['valores_unicos']}")
        print(f"    - Repetições: {distribuicoes[flag]['repeticoes']}")
    '''
    
    # Simular intervalo de tempo entre pacotes
    #time.sleep(0.5)  # Ajuste o tempo conforme necessário, 0.5 segundos
"""    

'\n# === PASSO 2: Simulação do Data Stream ===\nnum_iterations = 402513  # Número de iterações da simulação\n#Simula pacotes (amostras) chegando,e essas amostras sao extraidas do dataset.\n#Cada iteração do loop simula a chegada de um novo pacote (ou seja, uma nova amostra é processada).\nstream_results = []\n\nprint("Monitorando entropia das flags TCP...\n")\n\nfor i in range(num_iterations):\n    # Simular novos pacotes (extraindo amostras do conjunto de dados original)\n    #new_sample = df2.sample(n=1).iloc[0].to_dict() #A mesma amostra pode ser escolhida mais de uma vez.\n    #Em cada iteração, uma amostra é escolhida aleatoriamente do df2\n\n    new_sample = df2.iloc[i].to_dict()  #Simula novos pacotes (extraindo amostras em oderm sequencial do dataset)\n    entropy_monitor.update_stream(new_sample)\n    \n    # Calcular a entropia na janela atual\n    entropia_atual = entropy_monitor.calculate_entropy()\n    \n    # Armazenar os resultados do stream\n    #stream_results.append({

In [9]:
'''
# === PASSO 3: Criar DataFrame com os resultados do Data Stream ===
stream_df = pd.DataFrame(stream_results)

# Salvar os resultados em um CSV para análise posterior
stream_df.to_csv("dataset_entropia_legitimo_malicioso.csv", index=False)

print("\nMonitoramento finalizado! Resultados salvos em 'dataset_entropia_legitimo_malicioso'")
'''

'\n# === PASSO 3: Criar DataFrame com os resultados do Data Stream ===\nstream_df = pd.DataFrame(stream_results)\n\n# Salvar os resultados em um CSV para análise posterior\nstream_df.to_csv("dataset_entropia_legitimo_malicioso.csv", index=False)\n\nprint("\nMonitoramento finalizado! Resultados salvos em \'dataset_entropia_legitimo_malicioso\'")\n'

In [10]:
# === PASSO 4: Criar Gráfico ===

# Definir o número de amostras desejadas
#num_iterations = 880000  # Altere esse valor para controlar quantas amostras serão plotadas

# Preparar os dados para o gráfico
iterations = stream_df['Iteração']
entropia_ack = stream_df['tcp.flags.ack']
entropia_fin = stream_df['tcp.flags.fin']  
entropia_syn = stream_df['tcp.flags.syn']
entropia_urg = stream_df['tcp.flags.urg']
entropia_delta = stream_df['tcp.time_delta']

# Criar o gráfico
plt.figure(figsize=(10, 6))  # Tamanho do gráfico
plt.plot(iterations, entropia_ack, label='tcp.flags.ack', marker='o')
plt.plot(iterations, entropia_fin, label='tcp.flags.fin', marker='X')
plt.plot(iterations, entropia_syn, label='tcp.flags.syn', marker='^')
plt.plot(iterations, entropia_urg, label='tcp.flags.urg', marker='d')
plt.plot(iterations, entropia_delta, label='tcp.time_delta', marker='+')


# Adicionar título e rótulos
plt.title(f'Entropia das Flags TCP ao Longo das Iterações (Primeiras {num_iterations}Amostras)')
plt.xlabel('Iteração')
plt.ylabel('Entropia')
plt.legend()  # Mostrar legenda
plt.grid(True)  # Adicionar grid ao gráfico

# Mostrar o gráfico
plt.show()

KeyError: 'tcp.flags.urg'